#Introduction

My name is Samuel Hennon and my type of model is based on MultiNomial Naive Bayes using Bayes thoerm.It calculates probability and was modeled after the article Fake News Detection using NLP techniques by Joyce Annie George.



#IMPORTED LIBRARIES





In [ ]:
#Import libraries and connect to google drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')


MessageError: ignored

In [ ]:
#Import model specific resources or libraries
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics                                                 
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#The basepath is the same as what it is in the class assignments for simplicity
basePath = "/content/drive/My Drive/Colab Notebooks/Artificial Intelligence/Data/"

##Data files used, and how the files are structured

The Dataset used for this Machine Learning Model is a Kaggle Dataset consisting of Fake and Real news articles, formatted in 'csv' files. 

The link to the dataset is: https://www.kaggle.com/c/fake-news/data?select=test.csv

Downloading the dataset will give you two file "train.csv" and "test.csv". For simplicities sake, the contents of both datasets will get combined into one larger data set in which the Model will be trained and tested with.

As shown above, during the making of this Notebook the train and test csv files where loaded into the base path directory that was usued for all class assignment. This was done for consistancy with the class and for simplicity.

In [ ]:
# Data file name variables
train = basePath + "train.csv"
test = basePath + "test.csv"

In [ ]:
# To Delete
train = "train.csv"
test = "test.csv"

The CSV files come foramtted with 5 total columns. The columns are 'id', 'title', 'author', 'text', and 'label'. 

    ID: The designation number of the article in the file
    Title: Title of the associated news article
    Author: Person who wrote the News article
    Text: All words of the news article 
    Label: Designation of whether the news article is reliable or unreliable. O means unreliable, 1 means reliable. (These essentially refer to Fake or Real)

'Train.csv' has 20799 rows in it by default. 'Test.csv' has 25999 rows in it by default.

##Formatting the Data

In order to use data of the files, it has to be prepared and formatted in a certain manner. This invovles scrubbing the id, title, and author columns from the both datasets. Also, any rows with NAN values need to be removed entirely as they are unsuitable. 

The next step involves combining the contents of the train and test datasets into one final dataset that will be used for the Machine Learning portion of the notebook.

In [ ]:
# 0 = reliable, 1 = unreliable

# Read the training file
data_train = pd.read_csv(train)
data_train = data_train.drop(['id','author','title'], 1)
data_train.dropna( inplace=True) 
data_train.reset_index(inplace = True, drop = True)
print( data_train.shape )

# Read the testing data file
data_test  = pd.read_csv( test )
data_test = data_test.drop(['id','author','title'], 1)
data_test.dropna( inplace=True) 
data_test = data_test.loc[ :, :'label']
data_test.reset_index(inplace = True, drop = True)
print( data_test.shape )

# Combine both train and test datasets into a final dataset which is used later on
dataset = pd.concat( [data_train, data_test], ignore_index=True)
print( dataset.shape)
dataset.head()

#Check the final dataset to make sure it is valid for usage

The final dataset that will be used has 2 columns of "text" and "label". There are about 20762 total rows in the dataset, with about a 50:50 percent split between Fake and Real articles in the set. These statistics are shown below. 

In [ ]:
#graph the total number of fake and real articles in the final dataset
real_or_not = dataset.label.value_counts()
print( real_or_not )

plt.figure()
plt.bar( ['0','1'], real_or_not )
plt.show()

The dataset is lastly checked so that no NaN values are present. The ouptut of the next cell is empty because there are no NaN values present.

In [ ]:
dataset.dropna(inplace=True)
dataset[dataset.isnull().any(axis=1)]

##Supervised Machine Learning 

From this point on is where the actual Machine Learning and predictions of the Notebook will occur.

The next cell shows a helpful function to display the results of the machine learning in a discenrable manner, taken from this source.



In [ ]:
#A helpful function pulled from a website (credited at the end of this Notebook) that will print out the results of the Models processing in a concise and understandable manner.
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                  horizontalalignment="center",
                  color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

#Split the dataset into testing and training
The next cell shows the dataset being partitioned into training, and testing sets ( 80% for training, 20% for testing ) that will be used for the Model.

In [ ]:
y = dataset['label']

xtrain, xtest, ytrain, ytest = train_test_split( dataset['text'], y, test_size=0.2, random_state=7)

# Training Model

I am using a model called MultiNomial Naive Bayes model it uses bayes theorm that uses a naieve assumtion that all items are different and non related to eachother. This model was influenced by the mentioned article from Joyce Annie George the article is in the reference section at the end of the document.

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(data_train['text'],data_train['label'],test_size=0.2, random_state = 1)

data_train = data_train.fillna('')
data_train.replace([np.inf, -np.inf], np.nan, inplace=True)
data_train.fillna(999, inplace=True)


In [ ]:
data_train = data_train.fillna('')
pipe = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB())
])

model = pipe.fit(xtrain, ytrain)
prediction = model.predict(xtest)


#Model Results and Conclusions
This method is very effective with a accuracy of roughly 85% as shown in the plot out below.

In [ ]:
score = metrics.accuracy_score(ytest, prediction)
print("accuracy:   %0.3f" % (score*100))
cm = metrics.confusion_matrix(ytest, prediction, labels=[0,1])



fig, ax = plot_confusion_matrix(conf_mat=confusion_matrix(ytest, prediction),
                                show_absolute=True,
                                show_normed=True,
                                colorbar=True)
plt.show()

I lerned that the  MultiNomial Naive Bayes is an effective method that had a decent mix of accuracy and training speed.

Sources and References used:

https://iopscience.iop.org/article/10.1088/1742-6596/2007/1/012047/pdf

https://www.kaggle.com/c/fake-news/data?select=test.csv

https://towardsdatascience.com/detecting-fake-political-news-online-a571745f73dd

https://medium.com/analytics-vidhya/fake-news-detection-using-nlp-techniques-c2dc4be05f99

https://en.wikipedia.org/wiki/Bayes%27_theorem#:~:text=In%20probability%20theory%20and%20statistics,be%20related%20to%20the%20event.

